In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zlobin,HR,45.2924,14.6541,60.10,59,0,12.66,clear sky
1,1,Lobito,AO,-12.3481,13.5456,75.92,87,5,3.91,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,49.48,97,84,1.52,broken clouds
3,3,Grand Gaube,MU,-20.0064,57.6608,75.60,79,4,3.00,clear sky
4,4,Kyzyl-Suu,KG,42.3414,78.0056,55.96,49,0,5.46,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 60
What is the maximun temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zlobin,HR,45.2924,14.6541,60.10,59,0,12.66,clear sky
1,1,Lobito,AO,-12.3481,13.5456,75.92,87,5,3.91,clear sky
3,3,Grand Gaube,MU,-20.0064,57.6608,75.60,79,4,3.00,clear sky
6,6,Coahuayana,MX,18.7333,-103.6833,88.83,46,41,8.23,scattered clouds
10,10,Valparaiso,CL,-33.0393,-71.6273,61.03,77,100,6.91,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

398


City_ID                398
City                   398
Country                394
Lat                    398
Lng                    398
Max Temp               398
Humidity               398
Cloudiness             398
Wind Speed             398
Current Description    398
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

394


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zlobin,HR,45.2924,14.6541,60.10,59,0,12.66,clear sky
1,1,Lobito,AO,-12.3481,13.5456,75.92,87,5,3.91,clear sky
3,3,Grand Gaube,MU,-20.0064,57.6608,75.60,79,4,3.00,clear sky
6,6,Coahuayana,MX,18.7333,-103.6833,88.83,46,41,8.23,scattered clouds
10,10,Valparaiso,CL,-33.0393,-71.6273,61.03,77,100,6.91,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zlobin,HR,60.10,clear sky,45.2924,14.6541,
1,Lobito,AO,75.92,clear sky,-12.3481,13.5456,
3,Grand Gaube,MU,75.60,clear sky,-20.0064,57.6608,
6,Coahuayana,MX,88.83,scattered clouds,18.7333,-103.6833,
10,Valparaiso,CL,61.03,overcast clouds,-33.0393,-71.6273,
12,Muisne,EC,80.31,overcast clouds,0.6000,-80.0333,
14,Namatanai,PG,80.49,overcast clouds,-3.6667,152.4333,
16,Asosa,ET,68.31,overcast clouds,10.0667,34.5333,
17,Songea,TZ,65.50,overcast clouds,-10.6833,35.6500,
18,Jamestown,US,65.52,clear sky,42.0970,-79.2353,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zlobin,HR,60.10,clear sky,45.2924,14.6541,Hotel Bitoraj Fuzine
1,Lobito,AO,75.92,clear sky,-12.3481,13.5456,Casa Rosa Hotel Residence
3,Grand Gaube,MU,75.60,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
6,Coahuayana,MX,88.83,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
10,Valparaiso,CL,61.03,overcast clouds,-33.0393,-71.6273,Hotel Ultramar
12,Muisne,EC,80.31,overcast clouds,0.6000,-80.0333,Quinta Samay
14,Namatanai,PG,80.49,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
16,Asosa,ET,68.31,overcast clouds,10.0667,34.5333,Blendana Hotel Assosa
17,Songea,TZ,65.50,overcast clouds,-10.6833,35.6500,mtakuja
18,Jamestown,US,65.52,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))